# 문제 6

[Kaggle 형] train_prob.csv로 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col = 'id')
df_test = pd.read_csv('test_prob.csv', index_col = 'id')

In [3]:
df_train = df_train.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna()
)
df_test = df_test.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna()
)

In [5]:
df_train['product_code'].value_counts()

C    5765
E    5343
B    5250
A    5100
Name: product_code, dtype: int64

In [4]:
df_test['product_code'].value_counts()

D    5112
Name: product_code, dtype: int64

In [6]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer # , random_state=123
from sklearn.linear_model import LinearRegression

imp = IterativeImputer(
    estimator = LinearRegression(fit_intercept=True),
    random_state=123
)
X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']
df_train[X_imp] = df_train.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index = x.index, columns = X_imp)
)
df_test[X_imp] = df_test.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index = x.index, columns = X_imp)
)

In [16]:
X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
df_train[X_mean] = df_train.groupby('product_code')[X_mean].transform(
    lambda x: x.fillna(x.mean())
)
df_test[X_mean] = df_test.groupby('product_code')[X_mean].transform(
    lambda x: x.fillna(x.mean())
)

In [8]:
df_train['loading'] = df_train['loading'].fillna(df_train['loading'].mean())
df_test['loading'] = df_test['loading'].fillna(df_train['loading'].mean())

In [9]:
df_train['loading_log'] = np.log(df_train['loading'])
df_test['loading_log'] = np.log(df_test['loading'])

- LR, STD \['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17'\] 'na_1', 0.5917537024093409

- LDA, transform, predict

- LR + STD - PCA \['measurement_0 ~ _17'\] -> n_components = 7 # 0.581757510516433

- RF + {'max_depth': 7, 'min_samples_split': 512, 'n_estimators': 15} # 0.5745226991354744

In [17]:
X_all = df_test.columns.tolist()
df_train[X_all].isna().sum().sum(), df_test[X_all].isna().sum().sum()

(0, 0)

# Kaggle형 풀이 단계

Step 0: Kaggle용 데이터셋을 만든다.

Step 1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

Step 2: Baseline 모델을 만듭니다

Step 3: 모델 선택 루틴을 만듭니다.

|id|failure|
|----|----|
|16115| 0.1|
|16116| 0.2|

....	

Step 4: 모델 개선 작업을 합니다.

In [ ]:
# train / test 에서의 product_code가 등장하는 양상에 맞춰 검증 루틴을 구성해봅니다. (Step 1)

from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score

gkf = GroupKFold(4)
s_hist = list()

def eval_model(model_name, model):
    """
        모델의 인스턴스를 받아 검증 결과(AUC) 를 구합니다.
        1. GroupKFold 검증 - df_train, groups - product_code
        2. df_train 대한 교차검증결과에 대한 AUC를 구합니다.
        3. 주어진 모델명으로 평가 결과를 저장합니다. Format: Valid: {:.5f}±{:.5f}, Train: {:.5f}±{:.5f}
        4. 가장 최근의 수행결과를 보여 주어 선택하는데 활용하도록 합니다.
    Parameters:
        model_name: str, 모델의 이름,
        model: sklearn object, 모델 인스턴스
    """
    # TODO
    return result

# 모델 선택 루틴입니다. (Step 3)
def select_model(model):
    """
        1. 전체 학습데이터(df_train)로 학습을 합니다. 
        2. df_test에 대한 예측을 합니다.
        3. 예측 결과를 출력양식(id, failure)에 맞춰 csv파일을 만듭니다.
        4. 자가 채점을 위한 예측 결과를 반환합니다.
    Parameters: 
        model: sklearn object, 모델 인스턴스
    Returns: 1차원 np.ndarray
        예측 결과
    """
    # TODO
    return prd

In [25]:
from sklearn.model_selection import GroupKFold
gkf = GroupKFold(4)
for train_idx, valid_idx in gkf.split(df_train[X_all], df_train['failure'], groups = df_train['product_code']):
    df_cv_train, df_valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    print(
        df_cv_train['product_code'].unique(), df_valid['product_code'].unique()
    )

['A' 'B' 'E'] ['C']
['A' 'B' 'C'] ['E']
['A' 'C' 'E'] ['B']
['B' 'C' 'E'] ['A']


- LR, STD \['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17'\] 'na_1'

In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

ct = ColumnTransformer([
    ('std', StandardScaler(),  ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17']),
    ('pt', 'passthrough', ['na_1'])
])
clf_lr = make_pipeline(
    ct, LogisticRegression(solver = 'lbfgs')
)
cross_validate(
    clf_lr, df_train[X_all], df_train['failure'], groups = df_train['product_code'], scoring = 'roc_auc', 
    cv = gkf, return_train_score = True
)

{'fit_time': array([0.03463459, 0.03122592, 0.04008245, 0.0299015 ]),
 'score_time': array([0.00396538, 0.00530648, 0.        , 0.00974774]),
 'test_score': array([0.58822089, 0.58492694, 0.58894173, 0.59538985]),
 'train_score': array([0.59262299, 0.59350682, 0.59192443, 0.58956962])}